In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")
sys.path.append("../src")

In [3]:
from src.data.synth import get_dataset

train_ds, val_ds, input_dim, n_classes = get_dataset(batch_size=1, matter="white", path="../data")

In [4]:
from sklearn.ensemble import RandomForestClassifier
from common.metrics import get_val_metrics, get_tune_metrics
import yaml

with open("../config/model/acgan.yaml") as f:
    acgan_params = yaml.safe_load(f)
    del acgan_params["id"]

val_models = {"rf": RandomForestClassifier}
val_tune_metrics = get_tune_metrics("clf", train_ds[0].shape[1])
val_metrics = get_val_metrics("clf", False, "type", n_classes)

opt_params = {"id": "rmsprop", "learning_rate": 0.0008, "momentum": 0.05}

In [5]:
from src.models.acgan import ACGAN
from common.tfutils import get_optimizer

model = ACGAN(input_dim=input_dim, n_classes=n_classes, **acgan_params)
gen_optim = get_optimizer(**opt_params)
dis_optim = get_optimizer(**opt_params)

2024-05-14 01:48:39.305035: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-05-14 01:48:39.305059: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-05-14 01:48:39.305065: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-05-14 01:48:39.305086: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-14 01:48:39.305102: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
from src.train.synth import train_acgan

losses, scores, predictions, params = train_acgan(model, gen_optim, dis_optim, train_ds, 1, 1, val_models, val_ds, val_metrics, 1)

  0%|          | 0/272 [00:00<?, ?it/s]

EPOCH 1. Loss: D=4.95120, G=2.24445

rf_precision = 0.35661
rf_recall = 0.47009
rf_f1_score = 0.36407
rf_roc_auc = 0.62795



In [19]:
from sklearn.ensemble import RandomForestClassifier
from common.metrics import get_val_metrics, get_tune_metrics
import yaml

with open("../config/model/cvae.yaml") as f:
    cvae_params = yaml.safe_load(f)
    del cvae_params["id"]

val_models = {"rf": RandomForestClassifier}
val_tune_metrics = get_tune_metrics("clf", train_ds[0].shape[1])
val_metrics = get_val_metrics("clf", False, "type", n_classes)

opt_params = {"id": "rmsprop", "learning_rate": 0.0008, "momentum": 0.05}

In [20]:
from src.models.cvae import CVAE
from common.tfutils import get_optimizer

model = CVAE(input_dim=input_dim, n_classes=n_classes, **cvae_params)
optim = get_optimizer(**opt_params)

In [21]:
from src.train.synth import train_cvae

losses, scores, predictions, params = train_cvae(model, optim, train_ds, 1, 1, val_models, val_ds, 1)

  0%|          | 0/272 [00:00<?, ?it/s]

EPOCH 1. Loss = 0.17218



KeyboardInterrupt: 

In [18]:
scores

{'rf': [{'precision': 0.43951418951418947,
   'recall': 0.4700854700854701,
   'f1_score': 0.39135301091822833,
   'roc_auc': 0.6274093307428631},
  {'precision': 0.39409037684899756,
   'recall': 0.4444444444444444,
   'f1_score': 0.36708020486301035,
   'roc_auc': 0.614912830546854}]}